# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from citipy import citipy

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Puerto Natales,-51.7236,-72.4875,38.25,95,100,5.61,CL,1691688024
1,1,Marsá Maţrūḩ,31.3525,27.2453,80.94,62,0,12.84,EG,1691688581
2,2,Vilyuysk,63.7553,121.6247,61.16,63,0,7.94,RU,1691688099
3,3,Curitibanos,-27.2828,-50.5844,64.20,87,91,3.58,BR,1691688581
4,4,Zuyevka,58.4050,51.1335,67.86,72,2,4.65,RU,1691688581


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Distance",
    scale = 0.5,
    color = "Humidity",

) 


# Display the map plot
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = pd.DataFrame (city_data_df, columns = ["City", "Cloudiness", "Humidity", "Wind Speed", "Max Temp"])
ideal_city_df = ideal_city_df.loc[(ideal_city_df["Cloudiness"] == 0) & (ideal_city_df["Humidity"] < 50) & (ideal_city_df["Wind Speed"] < 5)]

# Drop any rows with null values
clean_ideal_city_df = ideal_city_df.dropna()

# Display sample data
clean_ideal_city_df

,City,Cloudiness,Humidity,Wind Speed,Max Temp
201,Prineville,0,48,0.00,75.61
293,Lerik,0,44,3.22,74.12
315,Cologne,0,46,2.30,78.42
386,Turpan,0,17,2.55,97.77
392,Bo`ka,0,25,4.32,81.10
464,Tamanghasset,0,10,3.44,94.91
527,Olivet,0,42,2.30,86.18


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame (city_data_df, columns = ["City", "Country", "Lat", "Lng", "Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Puerto Natales,CL,-51.7236,-72.4875,95,
1,Marsá Maţrūḩ,EG,31.3525,27.2453,62,
2,Vilyuysk,RU,63.7553,121.6247,63,
3,Curitibanos,BR,-27.2828,-50.5844,87,
4,Zuyevka,RU,58.4050,51.1335,72,
...,...,...,...,...,...,...
566,Moranbah,AU,-22.0016,148.0466,87,
567,Rakops,BW,-21.0167,24.3333,28,
568,Villa Gesell,AR,-37.2639,-56.9730,84,
569,Kanggye,KP,40.9695,126.5852,98,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Set parameters to search for a hotel
limit = 20
radius = 10000

params = {"categories" : "accomodation:hotel",
          "api_key": geoapify_key,
          "limit": limit}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
   
    # get latitude, longitude from the DataFrame
    
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    params["filter"] = f"circle:{Lng},{Lat}, {radius}"
    params["bias"] = f"proximity:{Lng},{Lat}" 
    
   
    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    name_address = response.json()    

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address[0]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
Puerto Natales - nearest hotel: No hotel found
Marsá Maţrūḩ - nearest hotel: No hotel found
Vilyuysk - nearest hotel: No hotel found
Curitibanos - nearest hotel: No hotel found
Zuyevka - nearest hotel: No hotel found
Vorgashor - nearest hotel: No hotel found
Sibolga - nearest hotel: No hotel found
Grytviken - nearest hotel: No hotel found
Ribeira Grande - nearest hotel: No hotel found
Labrador City - nearest hotel: No hotel found
Puerto Ayora - nearest hotel: No hotel found
Adamstown - nearest hotel: No hotel found
Lebu - nearest hotel: No hotel found
Mount Pearl - nearest hotel: No hotel found
Zhukovo - nearest hotel: No hotel found
Albany - nearest hotel: No hotel found
Topolobampo - nearest hotel: No hotel found
West Island - nearest hotel: No hotel found
Waitangi - nearest hotel: No hotel found
Egilsstadir - nearest hotel: No hotel found
Alofi - nearest hotel: No hotel found
Kapaa - nearest hotel: No hotel found
George Town - nearest hotel: No hotel found
Ust-

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Distance",
    scale = 0.5,
    color = "Hotel Name",

) 


# Display the map plot
map_plot_1